In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np
import tensorflow as tf

#To use all gpu-s
ctrl = Controller(*init_config())
mirrored_strategy = tf.distribute.MirroredStrategy()

2022-01-20 11:06:31.057030: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir


Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t11_zfmisc_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t78_zfmisc_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t35_relset_2


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


Eval episode has 25 steps and return -0.500.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-01-20 11:06:44.630798: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-20 11:06:44.631443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-20 11:06:45.897331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-20 11:06:45.897982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-20 11:06:45.898001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-20 11:06:45.899601: I

In [2]:
#Parsing meta datas
meta = ctrl.datasetManager._train_eps._meta
ds1 = []
for v in meta.values():
    ds1.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    if size>128:
        narr = narr[:128]
        size = 128
    return np.pad(narr, [1, 128-size], constant_values= [(299, 0)])


In [3]:
#episodes = ctrl.datasetManager._train_eps._storage
episodes = ctrl.datasetManager._train_eps._episodes
ds2 = []
#for pname, peps in episodes.items():
#    print(pname)
#    for length, eps in peps.items():
#        print(length, ':', len(eps), sep='\t')
     
for ep in episodes.values():
    for state in ep['text']:
        ds2.extend([str(s)[1:] for s in state])
        
ds = ds2 + ds1
parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [4]:
#This is a dataset which feeds goal strings
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x[:, :-1], x[:, 1:]
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

In [5]:
ds = iter(tf_ds)
next(ds)

2022-01-20 11:07:15.834803: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-20 11:07:15.853473: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz


(<tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[299,  96,  23, ..., 185, 187,   2],
        [299,   9,   5, ...,   0,   0,   0],
        [299, 231,   0, ...,   0,   0,   0],
        ...,
        [299,   9,   5, ..., 251, 252,  29],
        [299,  28,  23, ...,   0,   0,   0],
        [299, 231,   0, ...,   0,   0,   0]], dtype=int32)>,
 <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[ 96,  23,  13, ..., 187,   2,   9],
        [  9,   5,  15, ...,   0,   0,   0],
        [231,   0,   0, ...,   0,   0,   0],
        ...,
        [  9,   5, 237, ..., 252,  29,   7],
        [ 28,  23,  38, ...,   0,   0,   0],
        [231,   0,   0, ...,   0,   0,   0]], dtype=int32)>)

In [6]:
storage = ctrl.datasetManager._train_eps._storage
def extract_ep_names(eps):
    ep_names = []
    ep_length = []
    for k, v in eps.items():
        ep_names.extend([name for _, name in v])
        ep_length.extend([stat[2] for stat, name in v])
    ep_length = np.array(ep_length, dtype = np.float32)
    ep_length/=np.sum(ep_length)
    return (ep_names, ep_length)
refactored_storage = {name: extract_ep_names(eps) for name, eps in storage.items()}

In [7]:
refactored_storage

{'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1': (['logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T113619-2b4ab2aac2544615b253890f05919df4-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T113814-89e018c6005440338fb039fc9361d8d2-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T113916-d068175ad3584f7ebf7b1a72dc873342-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T113937-9bc1c6d8f6be4976926ea18f9412756a-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T113938-560704c7f77344f1af49ab2a7b89367e-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T114019-5c3a7b23844f4cc9a94476ad5e421490-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T114412-27f821acec0349b0b20c9ca3e648105c-26.npz',
   'logdir/train_eps/m2n140__t23_partfun1/m2n140__t23_partfun1-20211217T114446-ce41419f3eca

In [8]:
BATCH_SIZE_2 = 4
problems = list(storage.keys())
def generator2():
    for i in range(100):
        problem = np.random.choice(problems)
        ep_names, probs = refactored_storage[problem]
        eps = np.random.choice(ep_names, size = (BATCH_SIZE_2,), p = probs)
        res = []
        for name in eps:
            ep = episodes[name]
            length = len(ep['action'])
            q = np.random.randint(length)
            state = ep['text'][q]
            state = np.array([pad(np.array(parser.parse(str(goal)[1:]), dtype = np.int32)) for goal in state])
            state = np.pad(state[:128], [(0, 128-state.shape[0]), (0,0)])
            res.append(state)
        res = np.stack(res)
        yield ((res[:, :, :-1], res[:, :, 1:]), tf.zeros((1,), tf.int32))
        
sgn = tf.TensorSpec((BATCH_SIZE_2, 128, 128), dtype = tf.int32)
tf_ds2 = tf.data.Dataset.from_generator(generator2, output_signature = ((sgn, sgn), tf.TensorSpec((1,), tf.int32)))

In [9]:
sample = next(iter(tf_ds2))

In [10]:
from unorganized.parts5 import Net, Encoder, Decoder

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [11]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        self.loss_objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
    def call(self, y_true, y_pred):
        loss = self.loss_objective(y_true, y_pred)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        
        loss *= mask
        return loss

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=6000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [12]:
class StateNet(tf.keras.Model):
    def __init__(self, ):
        super().__init__()
        num_layers = 3
        embed_tokens = 512
        querry = 16
        self.goal_querry = querry
        length = 128
        d_model = 128
        dff = 512
        num_heads = 8
        dropout_rate = 0.1
        d_scale = 4
        self.goal_embedder = Net(num_layers, embed_tokens, querry, length, d_model, num_heads, dff, dropout_rate)
        self.latent_dense = tf.keras.layers.Dense(d_model*d_scale)
        self.state_encoder = Encoder(N = 3, querry = 16, output_length = 128, 
                                     d_model = d_model*d_scale, num_heads = num_heads, dff = dff*d_scale, rate = dropout_rate)
        self.state_decoder = Decoder(N = 3, output_length = 128, 
                                     d_model = d_model*d_scale, num_heads = num_heads, dff = dff*d_scale, rate = dropout_rate)
        self.out_dense = tf.keras.layers.Dense(d_model*self.goal_querry)
        
        self._loss = Loss()
        self._loss2 = tf.keras.losses.MeanSquaredError()
        lr = CustomSchedule(d_model, 1000)
        self._opt = tf.optimizers.Adam(lr, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
        
    def encode(self, inp, mask, training):
        batch_size = inp.shape[0]
        x = tf.reshape(inp, (128*batch_size, 128))
        goal_embed, goal_mask = self.goal_embedder.encode(x, training)
        #goal_embed = tf.reshape(x, (batch_size, 128) + (8, 128))
        x = tf.reshape(goal_embed, (batch_size, 128, self.goal_querry * 128))
        x = self.latent_dense(x)
        
        y, _ = self.state_encoder(x, mask, training)
        return y, goal_embed, goal_mask
    
    def decode(self, x, mask, training):
        x = self.state_decoder(x, mask, training)
        x = self.out_dense(x)
        x = tf.reshape(x, x.shape[:-1]+(self.goal_querry, 128))
        return x
        
    
    def calc_loss(self, inp, target, training):
        mask = tf.cast(tf.math.equal(inp[:, :, 1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        latent, goal_embed, goal_mask = self.encode(inp, mask, training)
        decoded_from_goals = self.goal_embedder.decode(goal_embed, goal_mask, training)
        decoded_from_goals = tf.reshape(decoded_from_goals, (4, 128,) + decoded_from_goals.shape[1:])
        goal_embed_reconst = self.decode(latent, mask, training)
        goal_embed_reconst = tf.reshape(goal_embed_reconst, (4*128,) + goal_embed_reconst.shape[2:])
        decoded_from_embed = self.goal_embedder.decode(goal_embed_reconst, goal_mask, training)
        decoded_from_embed = tf.reshape(decoded_from_embed, (4, 128,) + decoded_from_embed.shape[1:])
        
        losses = {}
        losses['goal_embedder'] = self._loss(target, decoded_from_goals)
        losses['goal_to_goal'] = self._loss(target, decoded_from_embed)
        losses['state_to_state'] = 0.2*self._loss2(goal_embed, goal_embed_reconst)
        return losses
    
    def call(self, x, training):
        inp, target = x
        losses = self.calc_loss(inp, target, training)
        scalars = {
                'goal_embedder': 2.5,
                'goal_to_goal': 0.2,
                'state_to_state': 0.1,
            }
        loss = 0.0
        for k, l in losses.items():
            loss += scalars[k]*l
        return loss
    
    @tf.function(input_signature=(sgn, sgn))
    def train(self, inp, target):
        with tf.GradientTape() as tape:
            losses = self.loss(inp, target, True)
            scalars = {
                'goal_embedder': 2.5,
                'goal_to_goal': 0.2,
                'state_to_state': 0.5,
            }
            loss = 0.0
            for k, l in losses.items():
                loss += scalars[k]*l
               
        return self._opt(tape, 4*loss, [self])
               
    

In [13]:
from unorganized.embedder_state import StateNet
model = StateNet()
goal_embedder = model.goal_embedder
num_layers = 5
embed_tokens = 512
querry = 16
length = 128
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

goal_embedder = model.goal_embedder#Net(num_layers, embed_tokens, querry, length, d_model, num_heads, dff, dropout_rate)

In [14]:
#learning_rate = CustomSchedule(d_model)
#optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
#goal_embedder.compile(optimizer = optimizer, loss=Loss(), metrics=[])

In [15]:
#goal_embedder.fit(tf_ds, epochs = 30)

In [16]:
learning_rate = CustomSchedule(512, 100)
optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer = optimizer, loss=lambda _, loss: loss, metrics=[])

In [17]:
sample = next(iter(tf_ds2))
inp, target = sample[0]
model.calc_loss(inp, target, False)

2022-01-20 11:07:16.573606: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-20 11:07:17.318044: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


{'goal_embedder': <tf.Tensor: shape=(), dtype=float32, numpy=0.042331874>,
 'goal_to_goal': <tf.Tensor: shape=(), dtype=float32, numpy=0.6148826>,
 'state_to_state': <tf.Tensor: shape=(), dtype=float32, numpy=1.4088563>}

In [23]:
model.fit(tf_ds2, epochs = 80)

Epoch 1/80
100/100 [==============================] - 34s 336ms/step - loss: 0.9198
Epoch 2/80
100/100 [==============================] - 34s 337ms/step - loss: 0.9664
Epoch 3/80
100/100 [==============================] - 34s 337ms/step - loss: 0.9776
Epoch 4/80
100/100 [==============================] - 34s 337ms/step - loss: 1.0795
Epoch 5/80
100/100 [==============================] - 34s 337ms/step - loss: 1.0184
Epoch 6/80
100/100 [==============================] - 34s 337ms/step - loss: 1.0886
Epoch 7/80
100/100 [==============================] - 34s 337ms/step - loss: 0.8944
Epoch 8/80
100/100 [==============================] - 34s 336ms/step - loss: 0.9641
Epoch 9/80
100/100 [==============================] - 34s 337ms/step - loss: 0.9238
Epoch 10/80
100/100 [==============================] - 34s 338ms/step - loss: 0.9467
Epoch 11/80
100/100 [==============================] - 34s 338ms/step - loss: 0.9670
Epoch 12/80
100/100 [==============================] - 34s 337ms/step - lo

KeyboardInterrupt: 

In [37]:
sample = next(iter(tf_ds2))
inp, target = sample[0]
model.calc_loss(inp, target, False)

{'goal_embedder': <tf.Tensor: shape=(), dtype=float32, numpy=0.29977998>,
 'goal_to_goal': <tf.Tensor: shape=(), dtype=float32, numpy=0.22830153>,
 'state_to_state': <tf.Tensor: shape=(), dtype=float32, numpy=0.00026369025>}

In [ ]:
inp, target = sample[0]
inp

In [38]:
self = model
training = False
mask = tf.cast(tf.math.equal(inp[:, :, 1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
latent, goal_embed, goal_mask = self.encode(inp, mask, training)
decoded_from_goals = self.goal_embedder.decode(goal_embed, goal_mask, training)
decoded_from_goals = tf.reshape(decoded_from_goals, (4, 128,) + decoded_from_goals.shape[1:])
goal_embed_reconst = self.decode(latent, mask, training)
goal_embed_reconst = tf.reshape(goal_embed_reconst, (4*128,) + goal_embed_reconst.shape[2:])
decoded_from_embed = self.goal_embedder.decode(goal_embed_reconst, goal_mask, training)
decoded_from_embed = tf.reshape(decoded_from_embed, (4, 128,) + decoded_from_embed.shape[1:])
        

In [41]:
goal_embed, goal_embed_reconst

(<tf.Tensor: shape=(512, 16, 128), dtype=float32, numpy=
 array([[[-1.5651295e+00,  1.0902956e-03,  1.3081441e+00, ...,
           3.6244854e-02,  1.8403567e-03,  6.7117810e-03],
         [-1.5651227e+00,  1.0840073e-03,  1.3081559e+00, ...,
           3.6248595e-02,  1.8387816e-03,  6.7123771e-03],
         [-1.5651294e+00,  1.0695681e-03,  1.3081644e+00, ...,
           3.6243409e-02,  1.8398734e-03,  6.7093670e-03],
         ...,
         [-1.5651300e+00,  1.0727122e-03,  1.3081396e+00, ...,
           3.6252365e-02,  1.8385561e-03,  6.7136772e-03],
         [-1.5651085e+00,  1.0750592e-03,  1.3081393e+00, ...,
           3.6252946e-02,  1.8355234e-03,  6.7113675e-03],
         [-1.5651133e+00,  1.0775849e-03,  1.3081458e+00, ...,
           3.6252484e-02,  1.8343662e-03,  6.7128465e-03]],
 
        [[-1.5651293e+00,  1.0902956e-03,  1.3081441e+00, ...,
           3.6244839e-02,  1.8403562e-03,  6.7117698e-03],
         [-1.5651228e+00,  1.0839999e-03,  1.3081561e+00, ...,
         

In [29]:
y_true = target
_y_pred = decoded_from_embed
_y_pred = tf.nn.softmax(_y_pred, axis=-1)
y_probs = tf.gather(_y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(_y_pred, axis=-1)
y_pred[:, :4, :8], y_true[:, :4, :8], tf.cast(100*y_probs[:, :4, :8], tf.int32)

(<tf.Tensor: shape=(4, 4, 8), dtype=int64, numpy=
 array([[[5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]],
 
        [[2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2]],
 
        [[2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2, 2, 2, 2]],
 
        [[3, 3, 3, 3, 3, 3, 3, 3],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]]])>,
 <tf.Tensor: shape=(4, 4, 8), dtype=int32, numpy=
 array([[[  9,   5,  12,  13,   2,   9,   5,  38],
         [  9,   5,  16,  43,   2,   9,   5, 306],
         [  9,   5,  18,  44,   2,   9,   5, 306],
         [  9,   5,  70,  71,   2,   9,   5, 303]],
 
        [[ 72,  23,  12,   2,  90,  23,   6,   7],
         [ 90,  23,  12,   2,  90,  23,   6,   7],
         [ 75,   5

In [39]:
y_true = target
_y_pred = decoded_from_embed
_y_pred = tf.nn.softmax(_y_pred, axis=-1)
y_probs = tf.gather(_y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(_y_pred, axis=-1)
y_pred[:, :4, :8], y_true[:, :4, :8], tf.cast(100*y_probs[:, :4, :8], tf.int32)

(<tf.Tensor: shape=(4, 4, 8), dtype=int64, numpy=
 array([[[5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]],
 
        [[2, 2, 2, 2, 2, 2, 2, 2],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]],
 
        [[5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]],
 
        [[2, 2, 2, 2, 2, 2, 2, 2],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5],
         [5, 5, 5, 5, 5, 5, 5, 5]]])>,
 <tf.Tensor: shape=(4, 4, 8), dtype=int32, numpy=
 array([[[  3,   9,   5, 180,   5, 180,   5,  19],
         [  9,   5,  16,  13,   2,   3,   9,   5],
         [  9,   5,  16, 180,   5, 180,   5, 180],
         [  9,   5,  43,  12,   2,   3,   4,   5]],
 
        [[  3,   9,   5, 180,   5,  15,   7,  10],
         [  0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0

In [ ]:
latent-latent[:, :1, :]

In [ ]:
decoded_goal_embed = model.decode(latent, mask, False)
decoded_goal_embed